### H3 - Existe algum tipo de relação entre produtos que gera uma compra "combinada"?

#### Importando pacotes

In [2]:
import pandas as pd

#### Lendo arquivos

In [8]:
arquivo = "e_commerce_pedidos.csv"

In [9]:
pedidos_df = pd.read_csv(arquivo)

In [10]:
pedidos_df.head()

,Unnamed: 0,id,data,data_pagamento,data_cancelamento,numero_pedido,status,pagamento,produto,sku,...,entrega_cidade,entrega_estado,entrega_cep,utm_source,utm_campaign,utm_medium,utm_content,utm_term,customizacao,id_cliente
0,1,117569438,04/08/2024 18:26:40,04/08/2024 18:27,NaN,1042552530523300,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Brasilia,Distrito Federal,72455490,NaN,NaN,NaN,NaN,NaN,NaN,31d69aed10
1,2,117385907,01/08/2024 19:08:02,01/08/2024 19:11,NaN,1042552893870166,Pagamento aprovado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
2,3,117385794,01/08/2024 19:05:53,NaN,01/08/2024 19:12,1042552190094688,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
3,4,117382873,01/08/2024 18:12:14,NaN,01/08/2024 19:13,1042552273445602,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5
4,5,117382771,01/08/2024 18:09:41,NaN,01/08/2024 19:13,1042552782939597,Cancelado,Pix,BRACELETE AJUSTÁVEL O COM COROA,DFREFSP8P,...,Varzea Nova(Santa Rita),Paraiba,58304500,NaN,NaN,NaN,NaN,NaN,NaN,3e109eedd5


#### Selecionando colunas relevantes
Esta tabela está a nível de sku. Assim temos para cada linha o sku que está incluso no pedido.

In [11]:
pedidos = pedidos_df[['numero_pedido', 'sku']]

In [12]:
pedidos.head()

,numero_pedido,sku
0,1042552530523300,DFREFSP8P
1,1042552893870166,DFREFSP8P
2,1042552190094688,DFREFSP8P
3,1042552273445602,DFREFSP8P
4,1042552782939597,DFREFSP8P


### Analisando produtos comprados em conjunto

#### Lista de produtos por pedido

In [13]:
df_produtos_por_pedido = pedidos.groupby('numero_pedido')['sku'].apply(list).reset_index()

In [14]:
df_produtos_por_pedido.head()

,numero_pedido,sku
0,1042552147040598,"[HL3J67375, 2QALTPNGA, 84WFBE5GN]"
1,1042552155461445,[JRM3YWN95]
2,1042552166282916,"[84WFBE5GN, 6XY5VVVRE]"
3,1042552190094688,[DFREFSP8P]
4,1042552221071163,"[JRM3YWN95, TMXCUUW38, E7SLSQLCC, HRPKRPNYD, T..."


#### Contabilizando produtos vendidos em conjunto

In [22]:
# Realiza o auto join para encontrar os produtos vendidos juntos no mesmo pedido
df_joined = pedidos.merge(pedidos, on="numero_pedido")

# Filtra as linhas onde os produtos são diferentes
df_joined = df_joined[df_joined['sku_x'] != df_joined['sku_y']]

# Agrupa e conta as combinações de produtos
df_resultado = df_joined.groupby(['sku_x', 'sku_y'])\
                        .agg(contagem=('numero_pedido', 'size'))\
                        .reset_index()

# Renomeia as colunas
df_resultado = df_resultado.rename(columns={'sku_x': 'produto', 'sku_y': 'produto_casado'})

# Ordena o resultado
df_resultado = df_resultado.sort_values(by=['contagem'], ascending=False)

In [23]:
df_joined.head()

,numero_pedido,sku_x,sku_y
17,1042552760701491,PPW5U8MVC,LSRE3YWF5
18,1042552760701491,LSRE3YWF5,PPW5U8MVC
23,1042552772676481,P7HXNQNUN,T5LJ2R3L6
24,1042552772676481,P7HXNQNUN,84WFBE5GN
25,1042552772676481,T5LJ2R3L6,P7HXNQNUN


In [27]:
df_resultado.head()

,produto,produto_casado,contagem
48,D56A2X2M7,F8ZW4347R,3
13,3LP9NDQHZ,D56A2X2M7,3
46,D56A2X2M7,3LP9NDQHZ,3
47,D56A2X2M7,74F6CQWEW,3
122,P7HXNQNUN,84WFBE5GN,3
